In [ ]:
import pandas as pd

## Import Raw Dataset Shop

In [ ]:
raw_data=pd.read_csv('dataset_shop.csv')
raw_data

In [ ]:
cm_seperate=raw_data['items'].apply(lambda x: x.split(','))
cm_seperate

# Convert String to Number

In [ ]:
item_name = ['Kemeja','Kaos','Jeans','Sepatu','Sendal']
item_no = [1, 2, 3, 4, 5]
recodes=dict(zip(item_name, item_no))

In [ ]:
listNumberedItems=[]
for i in range(len(cm_seperate)):
    numbered_list=list(map(lambda x:recodes[x],cm_seperate[i]))
    listNumberedItems.append(numbered_list)   
listNumberedItems

In [ ]:
data=raw_data.copy()
data['items']=listNumberedItems
data

# Create First C

In [ ]:
list_C, list_L = [], []

In [ ]:
def countItemSet(itemSet):
    itemSet['Support'] = 0
    for row in data['items']:
        for index, item in enumerate(itemSet['ItemSet']):
            if len(item) > len(row):
                continue
            if all(elem in row for elem in item):
                itemSet.loc[index, 'Support'] += 1
    return itemSet

In [ ]:
item_set = [item_no]
item_set = list(map(list, zip(*item_set)))
list_C.append(pd.DataFrame({'ItemSet':item_set}))
list_C[0] = countItemSet(list_C[0])
list_C[0]

In [ ]:
len(list_C[0])

# Create Secondary ItemSet

In [ ]:
minimumSupport = int(input())

In [ ]:
def removeLowestItemSet(C):
    global minimumSupport
    # minimumSupport = C['Support'].min()
    while C['Support'].min() <= minimumSupport:
        C = C.drop(C['Support'].idxmin())
    return C

In [ ]:
def findNextC(L):
    itemSetBefore = L['ItemSet']
    itemSet = []
    for i in range(len(L['ItemSet']) - 1):
        item = L['ItemSet'].iloc[i]
        for j in range(i+1, len(L['ItemSet'])):
            row = item.copy()
            for k in range(len(item)):
                if item[k] == L['ItemSet'].iloc[j][k] :
                    continue
                elif k == len(item) - 1:
                    if item[k] < L['ItemSet'].iloc[j][k]:
                        row.append(L['ItemSet'].iloc[j][k])
                else:
                    break
            if len(row) == len(item) + 1:
                itemSet.append(row)
    newItemSet = pd.DataFrame({'ItemSet':itemSet})
    return newItemSet

In [ ]:
def validate_item_set(C):
    validated_item_set = []
    for row in C['ItemSet']:
        #print(row)
        for numberedItem in listNumberedItems:
            if all(elem in numberedItem for elem in row):   
                validated_item_set.append(row)
                break
    newItemSet = pd.DataFrame({'ItemSet':validated_item_set})
    return newItemSet

In [ ]:
while True:
    new_L = removeLowestItemSet(list_C[-1])
    if len(new_L) <= 1:
        break
    list_L.append(new_L)
    #print(list_L[-1])
    
    new_C = findNextC(list_L[-1])
    new_C = validate_item_set(new_C)
    new_C = countItemSet(new_C)
    #print(new_C)
    list_C.append(new_C)

In [ ]:
print(list_C[-1])